In [1]:
import sys
sys.path.append('/home/alexj/hgcal-dev')

In [2]:
from hgcal_dev.visualization.hgcal import Event
import numpy as np
import pandas as pd
import torch
from hgcal_dev.utils.experiment import Experiment
from pathlib import Path
from hgcal_dev.clustering.meanshift import MeanShift
from tqdm import tqdm

In [4]:
run_path = Path('/home/alexj/outputs/wandb/run-20210119_182022-2bhsqvop')
ckpt_path = Path('/home/alexj/outputs/wandb/run-20210119_182022-2bhsqvop/files/hgcal-spvcnn/2bhsqvop/checkpoints/epoch=14-v1.ckpt')
experiment = Experiment(run_path, ckpt_path)

ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: alexschuy (use `wandb login --relogin` to force relogin)


In [5]:
def get_events(split, n=-1):
    if split == 'train':
        dataloader = experiment.datamodule.train_dataloader()
    elif split == 'val':
        dataloader = experiment.datamodule.val_dataloader()
    elif split == 'test':
        dataloader = experiment.datamodule.test_dataloader()
    if n == -1:
        input_paths = dataloader.dataset.events
    else:
        input_paths = dataloader.dataset.events[:n]
    events = []
    for input_path in tqdm(input_paths):
        pred_dir = experiment.run_prediction_dir / split
        event_name = input_path.stem
        pred_path = pred_dir / f'{event_name}.npz'
        events.append(Event(input_path, pred_path))
    return events


In [6]:
events = get_events('train', n=100)

100%|██████████| 100/100 [00:00<00:00, 106.10it/s]


In [15]:
np.unique(events[0].pred_class_labels, return_counts=True)

(array([0, 1, 2]), array([19691,    46,  2009]))

In [10]:
np.unique(events[0].input_event['labels_s'], return_counts=True)

(array([0, 1, 2]), array([19492,   325,  1929]))

In [7]:
clusterer = MeanShift(use_gpu=True)
event = events[0]
pred_instance_labels = clusterer.cluster(event.embedding)

In [7]:
sqs = np.zeros(len(events))
rqs = np.zeros(len(events))
pqs = np.zeros(len(events))
for i, event in tqdm(enumerate(events)):
    clusterer = MeanShift(use_gpu=True)
    pred_instance_labels = clusterer.cluster(event.embedding)
    sq_, rq_, pq_ = event.pq(pred_instance_labels)
    rqs[i] = rq_
    pqs[i] = pq_

0it [00:18, ?it/s]


KeyboardInterrupt: 

In [2]:
print(f'mean sq: {np.mean(sqs)}')
print(f'mean rq: {np.mean(rqs)}')
print(f'mean pq: {np.mean(pqs)}')

NameError: name 'np' is not defined

In [ ]:
Event.plot_confusion_matrix()

# Typical Events

In [ ]:
events[100].plot_event(semantic=True, truth=True)

In [ ]:
events[100].plot_event(semantic=True, truth=False)

In [ ]:
events[100].plot_event(semantic=False, truth=True)

In [ ]:
events[100].plot_event(semantic=False, truth=False)

In [ ]:
fig = events[100].plot_3d(semantic=True, truth=True)

In [ ]:
fig = events[100].plot_3d(semantic=True, truth=False)

In [ ]:
fig = events[100].plot_3d(semantic=False, truth=True)

In [ ]:
fig = events[100].plot_3d(semantic=False, truth=False)